#### 第十六章 协程
##### 16.1 生成器如何进化成协程
- yield 关键字可以在表达式中使用，而且生成器 API 中增加了 .send(value) 方法。生成器的调用方可以使用 .send(...) 方法发送数据，发送的数据会成为生成器函数中 yield 表达式的值。因此，生成器可以作为协程使用。协程是指一个过程，这个过程与调用方协作，产出由调用方提供的值。
- 除了 .send(...) 方法，PEP 342 还添加了 .throw(...) 和 .close()方法：前者的作用是让调用方抛出异常，在生成器中处理；后者的作用是终止生成器。

##### 16.2 用作协程的生成器的基本行为

In [7]:
def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:', x)

In [23]:
import inspect
my_coro = simple_coroutine()
print(my_coro,inspect.getgeneratorstate(my_coro))
print(next(my_coro), inspect.getgeneratorstate(my_coro))
my_coro.send(42)

<generator object simple_coroutine at 0x0000025B208F4CF0> GEN_CREATED
-> coroutine started
None GEN_SUSPENDED
-> coroutine received: 42


StopIteration: 

In [24]:
# 产出两个值的协程
def simple_coro2(a):
    print('-> Startd: a =', a)
    b = yield a
    print('-> Recevied: b =', b)
    c = yield a + b
    print('-> Recevied: c =', c)

In [31]:
my_coro2 = simple_coro2(14)
from inspect import getgeneratorstate

print(getgeneratorstate(my_coro2))
print(next(my_coro2))
print(getgeneratorstate(my_coro2))
print(my_coro2.send(28))
print(my_coro2.send(99))

GEN_CREATED
-> Startd: a = 14
14
GEN_SUSPENDED
-> Recevied: b = 28
42
-> Recevied: c = 99


StopIteration: 

In [33]:
print(getgeneratorstate(my_coro2))

GEN_CLOSED


##### 16.3 使用协程计算移动平均值

In [47]:
# 定义一个计算移动平均值的协程
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [41]:
coro_avg = averager()
print(next(coro_avg))
print(coro_avg.send(10))
print(coro_avg.send(30))
print(coro_avg.send(5))

None
10.0
20.0
15.0


##### 16.4 预激协程的装饰器
如果不预激，那么协程没什么用。为了简化协程的用法，有时会使用一个预激装饰器。

In [50]:
# 预激协程的装饰器
from functools import wraps

def coroutine(func):
    @wraps(func)
    def wrimer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return wrimer

In [51]:
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [55]:
coro_avg = averager()
from inspect import getgeneratorstate
print(getgeneratorstate(coro_avg))
print(coro_avg.send(10))
print(coro_avg.send(30))
print(coro_avg.send(15))

GEN_SUSPENDED
10.0
20.0
18.333333333333332


##### 16.5 终止协程和异常处理
协程中未处理的异常会向上冒泡，传给 next 函数或 send 方法的调用方（即触发协程的对象）。

In [58]:
# 为处理的异常会导致协程终止
coro_avg = averager()
print(coro_avg.send(40))
print(coro_avg.send(45))
coro_avg.send('spam')

40.0
42.5


TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [60]:
coro_avg.send(60)

StopIteration: 

In [62]:
# 学习在协程中处理异常的测试代码
class DemoException(Exception):
    '''为这次演示定义的异常类型'''
    
def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else:
            print('-> coroutine recevied: {!r}'.format(x))
    raise RuntimeError('This line should never run.')

In [69]:
# 激活和关闭demo_exc_handling，没有异常
exc_coro = demo_exc_handling()
next(exc_coro)
exc_coro.send(11)
exc_coro.send(22)
exc_coro.close()
from inspect import getgeneratorstate
print(getgeneratorstate(exc_coro))

-> coroutine started
-> coroutine recevied: 11
-> coroutine recevied: 22
GEN_CLOSED


In [74]:
# 把DemoException异常传入demo_exc_handling不会导致协程中止
exc_coro = demo_exc_handling()
next(exc_coro)
exc_coro.send(11)
exc_coro.throw(DemoException)
print(getgeneratorstate(exc_coro))

-> coroutine started
-> coroutine recevied: 11
*** DemoException handled. Continuing...
GEN_SUSPENDED


In [82]:
# 如果无法处理传入的异常，协程会中止
exc_coro = demo_exc_handling()
next(exc_coro)
exc_coro.send(11)
exc_coro.throw(ZeroDivisionError)

-> coroutine started
-> coroutine recevied: 11
-> coroutine ending


ZeroDivisionError: 

In [80]:
print(getgeneratorstate(exc_coro))

GEN_CLOSED


In [81]:
# 如果不管协程如何结束都想做些清理工作，要把协程定义体重相关的代码放入try/finally块中。
class DemoException(Exception):
    '''为这次演示定义的异常类型'''
    
def demo_exc_handling():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...')
            else:
                print('-> coroutine recevied: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

##### 16.6 让协程返回值

In [83]:
# 定义一个球平均值的协程，让它返回一个结果
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return Result(count, average)

In [90]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(45)
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
print(result)

Result(count=3, average=28.333333333333332)


##### 16.7 使用yield from

In [100]:
# 使用yield from 计算平均值并输出统计报告
from collections import namedtuple

Result = namedtuple('Result', 'count average')

# 子生成器
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return Result(count, average)

# 委派生成器
def grouper(results, key):
    while True:
        results[key] = yield from averager()

# 客户端代码，即调用方
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)
    print(results)
    report(results)

# 输出报告
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} average {:.2f}{}'.format(
            result.count, group, result.average, unit))

In [101]:
data = {
    'girls;kg':
    [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
    [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
    [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
    [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}
main(data)

{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997), 'boys;kg': Result(count=9, average=40.422222222222224), 'boys;m': Result(count=9, average=1.3888888888888888)}
 9 boys  average 40.42kg
 9 boys  average 1.39m
10 girls average 42.04kg
10 girls average 1.43m


##### 16.8 yield from 的意义
##### 16.9 使用协程做离散事件仿真

#### 第十七章 使用期物处理并发
##### 17.1 网络下载的三种风格

In [25]:
# 依序下载的脚本
import os
import time
import sys

import requests

POP20_CC = 'CN IN US ID BR PK NG BD RU JP \
            MX PH VN ET EG DE IR TR CD FR'.split()

BASE_URL = 'http://flupy.org/data/flags'

DEST_DIR = 'downloads/'


def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)


def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = requests.get(url)
    return resp.content


def show(text):
    print(text, end=' ')
    sys.stdout.flush()


def download_many(cc_list):
    for cc in sorted(cc_list):
        image = get_flag(cc)
        show(cc)
        save_flag(image, cc.lower() + '.gif')
    return len(cc_list)


def main(download_many):
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))

In [26]:
main(download_many)

BD BR CD CN DE EG ET FR ID IN 

KeyboardInterrupt: 

In [27]:
# 使用futures.ThreadPoolExecutor类实现多线程下载
from concurrent import futures

MAX_WORKERS = 20

def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc


def download_many(cc_list):
    workers = min(MAX_WORKERS, len(cc_list))
    with futures.ThreadPoolExecutor(workers) as executor:
        res = executor.map(download_one, sorted(cc_list))
    return len(list(res))

In [28]:
main(download_many)

BD DE ID BR CN VN TR PK NG PH ET FR IN JP MX RU EG CD IR US 
20 flags downloaded in 3.02s


In [13]:
# 把download_many函数中的executor.map方法缓存executor.submit方法和futures.as_completed函数
def download_many(cc_list):
    cc_list = cc_list[:5]
    with futures.ThreadPoolExecutor(max_workers=3) as executor:
        to_do = []
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc)
            to_do.append(future)
            msg = 'Scheduled for {}: {}'
            print(msg.format(cc, future))
        
        results = []
        for future in futures.as_completed(to_do):
            res = future.result()
            msg = '{} result: {!r}'
            print(msg.format(future, res))
            results.append(res)
            
    return len(results)

In [15]:
main(download_many)

Scheduled for BR: <Future at 0x1fb4fe92ac8 state=running>
Scheduled for CN: <Future at 0x1fb4fe95588 state=running>
Scheduled for ID: <Future at 0x1fb4feb7f60 state=running>
Scheduled for IN: <Future at 0x1fb4fec2da0 state=pending>
Scheduled for US: <Future at 0x1fb4feb80b8 state=pending>
IDBR  <Future at 0x1fb4fe92ac8 state=finished returned str> result: 'BR'
<Future at 0x1fb4feb7f60 state=finished returned str> result: 'ID'
US <Future at 0x1fb4feb80b8 state=finished returned str> result: 'US'
CN <Future at 0x1fb4fe95588 state=finished returned str> result: 'CN'
IN <Future at 0x1fb4fec2da0 state=finished returned str> result: 'IN'

5 flags downloaded in 0.91s


##### 17.3 使用concurrent.futures模块启动进程
- 这个模块实现的是真正的并行计算，因为它使用 ProcessPoolExecutor 类把工作分配给多个Python 进程处理。因此，如果需要做 CPU 密集型处理，使用这个模块能绕开 GIL，利用所有可用的 CPU 核心。
- ProcessPoolExecutor 和 ThreadPoolExecutor 类都实现了通用的Executor 接口，因此使用 concurrent.futures 模块能特别轻松地把基于线程的方案转成基于进程的方案。

##### 17.4 实验Executor.map方法

In [23]:
# 简单演示ThreadPoolExecutor类的map方法
from time import sleep, strftime
from concurrent import futures


def display(*args):
    print(strftime('[%H:%M:%S]'), end=' ')
    print(*args)


def loiter(n):
    msg = '{}loiter({}): doing nothing for {}s...'
    display(msg.format('\t'*n, n, n))
    sleep(n+1)
    msg = '{}loiter({}): done.'
    display(msg.format('\t'*n, n))
    return n * 10


def main():
    display('Script starting.')
    executor = futures.ThreadPoolExecutor(max_workers=3)
    results = executor.map(loiter, range(5))
    display('results:', results)
    display('Waiting for individual results:')
    for i, result in enumerate(results):
        display('result {}: {}'.format(i, result))

In [25]:
main()

[15:43:12] Script starting.
[15:43:12] loiter(0): doing nothing for 0s...
[15:43:12][15:43:12][15:43:12] 	loiter(1): doing nothing for 1s...
 		loiter(2): doing nothing for 2s...
 results: <generator object Executor.map.<locals>.result_iterator at 0x0000014E3C34BB88>
[15:43:12] Waiting for individual results:
[15:43:13] loiter(0): done.
[15:43:13][15:43:13] 			loiter(3): doing nothing for 3s...
 result 0: 0
[15:43:14] 	loiter(1): done.
[15:43:14][15:43:14] result 1: 10
 				loiter(4): doing nothing for 4s...
[15:43:15] 		loiter(2): done.
[15:43:15] result 2: 20
[15:43:17] 			loiter(3): done.
[15:43:17] result 3: 30
[15:43:19] 				loiter(4): done.
[15:43:19] result 4: 40


##### 17.5 显示下载进度并处理错误

In [30]:
import time
from tqdm import tqdm
for i in tqdm(range(1000)):
    time.sleep(.01)

100%|██████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 91.57it/s]


In [32]:
def get_flag(base_url, cc):
    url = '{}/{cc}/{cc}.gif'.format(base_url, cc=cc.lower())
    resp = requests.get(url)
    if resp.status_code != 200:
        resp.raise_for_status()
    return resp.content


def download_one(cc, base_url, verbose=False):
    try:
        image = get_flag(base_url, cc)
    except requests.exceptions.HTTPError as exc:
        res = exc.response
        if res.status_code == 404:
            status = HTTPStatus.not_found
            msg = 'not found'
        else:
            raise
    else:
        save_flag(image, cc.lower() + '.gif')
        status = HTTPStatus.ok
        msg = 'OK'
    
    if verbose:
        print(cc, msg)
    
    return Result(status, cc)

In [33]:
def download_many(cc_list, base_url, verbose, max_req):
    counter = collections.Counter()
    cc_iter = sorted(cc_list)
    if not verbose:
        cc_iter = tqdm.tqdm(cc_iter)
    for cc in cc_iter:
        try:
            res = download_one(cc, base_url, verbose)
        except requests.exceptions.HTTPError as exc:
            error_msg = 'HTTP error {res.status_code} - {res.reason}'
            error_msg = error_msgmsg.format(res=exc.response)
        except requests.exceptions.ConnectionError as exc:
            error_msg = 'Connection error'
        else:
            error_msg = ''
            status = res.status

        if error_msg:
            status = HTTPError.error
        counter[status] += 1
        if verbose and error_msg:
            print('*** Error for {}: {}'.format(cc, error_msg))
    return counter